In [2]:
from get_summoner_data import load_clean_df

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from collab_filter import ChampionsDataset, DotProduct, train_and_evaluate_model
from torch import optim

In [5]:
df = load_clean_df()

# train test split
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = ChampionsDataset(train_df)
test_dataset = ChampionsDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader= DataLoader(test_dataset, batch_size=10, shuffle=True)

num_summoners = len(df['puuid'].unique())
num_champions = len(df['champ_id'].unique())
num_factors = 5

In [7]:
model = DotProduct(num_summoners, num_champions, num_factors)
train_and_evaluate_model(model, train_loader, test_loader)

Epoch 1, Loss: 1121.1969470793642
Test MSE (Rounded Predictions): 1192.6302490234375
Epoch 2, Loss: 1121.3209521395022
Test MSE (Rounded Predictions): 1192.572998046875
Epoch 3, Loss: 1121.4735488551935
Test MSE (Rounded Predictions): 1192.65283203125
Epoch 4, Loss: 1120.870343339156
Test MSE (Rounded Predictions): 1192.657470703125
Epoch 5, Loss: 1120.7608634120818
Test MSE (Rounded Predictions): 1192.692138671875
Epoch 6, Loss: 1120.6502414604008
Test MSE (Rounded Predictions): 1192.704345703125
Epoch 7, Loss: 1121.7747355843073
Test MSE (Rounded Predictions): 1192.7005615234375
Epoch 8, Loss: 1120.4307819260014
Test MSE (Rounded Predictions): 1192.7010498046875
Epoch 9, Loss: 1121.1466955818303
Test MSE (Rounded Predictions): 1192.7059326171875
Epoch 10, Loss: 1120.2116433781664
Test MSE (Rounded Predictions): 1192.6011962890625


In [8]:
all_champions = torch.tensor([i for i in range(num_champions)])
user_id = torch.tensor([1] * len(all_champions)) 
predicted_ratings = model(user_id, all_champions)
predicted_ratings

tensor([0.3008, 0.2638, 0.3812, 0.0864, 0.7844, 0.6236, 0.7397, 0.6504, 0.8032,
        0.3481, 0.5365, 0.7820, 0.9216, 0.0155, 0.7607, 0.8119, 0.2666, 0.6065,
        0.2259, 0.5666, 0.1568, 0.3313, 0.0822, 0.9495, 0.7929, 0.1126, 0.6142,
        0.5428, 0.0317, 0.9363, 0.6708, 0.9401, 0.1017, 0.0441, 0.4146, 0.3290,
        0.5985, 0.2450, 0.3012, 0.2924, 0.7246, 0.5202, 0.3780, 0.6526, 0.5114,
        0.0583, 0.4743, 0.1467, 0.6930, 0.6873, 0.7056, 0.6850, 0.7330, 0.7981,
        0.8357, 0.9371, 0.2611, 0.8093, 0.6916, 0.7574, 0.1950, 0.3517, 0.1980,
        0.3890, 0.7409, 0.2443, 0.9373, 0.3716, 0.7729, 0.8869, 0.8959, 0.7206,
        0.0163, 0.8036, 0.6164, 0.6283, 0.3077, 0.4292, 0.1727, 0.1463, 0.2789,
        0.7222, 0.7036, 0.7319, 0.3844, 0.8932, 0.5288, 0.3440, 0.3986, 0.4271,
        0.6894, 0.2701, 0.9175, 0.7554, 0.5757, 0.6989, 0.5530, 0.2524, 0.8336,
        0.2219, 0.9855, 0.7482, 0.8805, 0.5739, 0.3331, 0.3225, 0.6991, 0.4370,
        0.5150, 0.4218, 0.9889, 0.1558, 

In [ ]:
"""
for epoch in range(epochs):
    # set in training mode
    model.train()
    # some metric of keeping track of losses
    loss = 0
    criterion = nn.MSELoss()
    # these are tensors, in this case the locations of whichever data batch is being handed over
    for user_id, champ_id, ratings in train_loader:
        # Just empties the gradients before
        optimizer.zero_grad()
        # this gets passed into the forward function
        preds = model(user_id, champ_id)
        # compares yhat to y
        loss = criterion(preds, ratings)

        # Computes the dloss/dx for every parameter x... what is x?... so something like x.grad += dloss/dx 
        loss.backward()
        # updates the x value using x.grad... so x += -lr * x.grad
        optimizer.step()
        loss += loss.item()

        
        
        
        
        
Synapse that transforms the X into something... and each node then puts an activation function (think sigmoid)
Think of this as the W in the matmul, except this time we're literally just amking it a... dot product?
gradient descent is saying do we want W to go... up or down? gives us a direction to minimize cost
Back prop is essentially saying that we are starting from the end and the "error" term of gradient descent is passed on to the node which contributes the most
every example gets a vote on which way is downhill        
"""

One layer
    Synapse takes the dot product 

Main confusing parts are loss.backward() and optimizer.step() (what are the params excatly?) as well as what's going on in the forward.... like what are user_id and champ_id representing? 

In [ ]:
import json
import os
import pickle as pkl
import time
from collections import defaultdict
from typing import Any, Dict, List, Optional

import pandas as pd
import requests
from sklearn.preprocessing import LabelEncoder

from get_champ_id_mapping import get_champ_id_to_name

API_KEY = os.getenv("RIOT_API_KEY")
PROJECT_ROOT = "/Users/brtnl/OneDrive/Desktop/code/lol_main_recommender"
HEADERS = {"X-Riot-Token": API_KEY}
REGION = "na1"
MATCH_REGION = "americas"

pkl_path = os.path.join(PROJECT_ROOT, "src/data.pkl")

json_folder_path = os.path.join(PROJECT_ROOT, "src/summoner_data/")

player_champions = defaultdict(
    lambda: defaultdict(int)
)  # {player_id: {champion_id: count}}
for json_file in os.listdir(json_folder_path):
    puuid = json_file.split(".json")[0]
    with open(os.path.join(json_folder_path, json_file), "r") as f:
        match_data = json.load(f)
    for match in match_data:
        participants = match.get("info", {}).get("participants", [])
        for participant in participants:
            if participant.get("puuid") == puuid:
                # Store information to get rating
                champion_id = participant.get("championId")
                win = participant.get("win")
                kills = participant.get("kills")
                deaths = participant.get("deaths")
                assists = participant.get("assists")

                # rating = (win_percentage_weight * win_percentage) + (kda_weight * kda) + (matches_played_weight * matches_played)
                player_champions[puuid][champion_id] += 1
                player_champions[puuid][f"{champion_id}_kills"] += kills
                player_champions[puuid][f"{champion_id}_assists"] += assists
                player_champions[puuid][f"{champion_id}_deaths"] += deaths
                if win == 1:
                    player_champions[puuid][f"{champion_id}_wins"] += 1

In [26]:
player_champion_stats = defaultdict(lambda: defaultdict(int))
# calculate kda
for puuid, champ_data in player_champions.items():
    for champion_id, count in champ_data.items():
        # If the key is equal to the id, can construct the stats
        if isinstance(champion_id, int):
            kills = player_champions[puuid][f"{champion_id}_kills"]
            assist = player_champions[puuid][f"{champion_id}_assists"]
            deaths = player_champions[puuid][f"{champion_id}_deaths"]
            wins = player_champions[puuid].get(f"{champion_id}_wins", 0)
            win_pct = (wins / count) * 100 if count > 0 else 0
            kda = (kills + assists) / (deaths if deaths != 0 else 1)

            player_champion_stats[puuid][champion_id] = kda + win_pct + count

In [27]:
player_champion_stats

defaultdict(<function __main__.<lambda>()>,
            {'-zSba7M6_SDDJD1sJ3n2jYDih8mLAawHOXA-hNB554wLALfsNcCPUsrEyaB-7qg8MY3rLQhYQB0KmQ': defaultdict(int,
                         {55: 3.0,
                          84: 3.0769230769230766,
                          235: 103.22222222222223,
                          8: 102.55555555555556,
                          67: 75.1661750245821,
                          92: 109.57157640565711,
                          18: 6.0,
                          126: 31.272727272727273,
                          5: 54.45454545454545,
                          221: 12.0,
                          98: 102.2,
                          91: 101.71428571428571,
                          21: 104.55555555555556,
                          523: 111.5,
                          64: 8.5,
                          238: 102.5,
                          37: 101.5,
                          266: 109.5,
                          51: 7.0,
                          76: 3.